Build a Classification Tree with Gini index
There are five steps to follow
<br>
1.Build Gini index
<br>
2.create split
<br>
    Based on a attribute, separate the dataset, and build a dictionary to store the information about this split
    <br>
3.create a tree with dictionary
<br>
    left:{}
    <br>
    right:{}
    <br>
    1)how to determine the terminal node
    <br>
        1.if the depth reach the maximum
        <br>
        2.one of child nodes is empty
        <br>
        3.minimum node records, if the records of this node is too small, we can't split
    <br>
    2)how to do recursive splitting
    <br>
    3)build a tree with left and right node
    <br>
    


In [8]:
import numpy as np
import heapq
import pandas

#A class that used to store decision tree
class CARTree:
    def __init__(self):
        self.tree = None
        self.tree_depth = 0
    #Print the tree 
    def __str__(self):
        print(self.tree)
        
    def learn(self, train_dataset, max_depth, min_size):
        root = self.node_finding(train_dataset)
        self.tree_depth = 1
        self.tree = root
        self.split(self.tree,max_depth,min_size)
        
    def split(self, node, max_depth, min_size):
        left, right = node['groups']
        del(node['groups'])
        self.tree_depth += 1
        #Case 1
        if self.tree_depth >= max_depth:
            #create terminal value for left leaf node
            node['left'] = self.to_terminal(left)
            #create terminal value for right leaf node
            node['right'] = self.to_terminal(right)
            return
        #Case 2
        if not left or not right:
            node['left']=node['right'] = self.to_terminal(left+right)
            return
        #Case 3
        if len(left) <= min_size:
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.node_finding(left)
            self.split(node['left'], max_depth, min_size)

        if len(right) <= min_size:
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.node_finding(right)
            self.split(node['right'], max_depth, min_size)        
    
    #Find the best node
    def node_finding(self,dataset):
        class_values = list(set([row[-1] for row in dataset]))
        b_attribute, b_threshold, b_groups = 999,999,None
        score = 999
        print("Total number of attributes is %r" %len(dataset[0]))
        for attribute in range(len(dataset[0])-1):
            for row in dataset:
                #try to find a suitable threshold to separate the data 
                threshold = row[attribute]
                groups = self.split_dataset(dataset,attribute,threshold)
                current_score = self.gini_index(groups, class_values)
                #print('X%d < %.3f Gini=%.3f' % ((attribute+1), row[attribute], current_score))
                if current_score < score:
                    score = current_score
                    b_attribute = attribute
                    b_threshold = threshold
                    b_groups = groups
                else:
                    pass
        return {'attribute':b_attribute, 'threshold':b_threshold, 'score':score,
                'groups': b_groups}
    
    #Give a gini score of a split
    def gini_index(self, groups, class_values):
        total_instance = float(sum([len(group) for group in groups]))
        gini = 0
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            score = 0
            for value in class_values:
                proportion = [row[-1] for row in group].count(value) / float(size)
                score += proportion * proportion
            gini += (1.0 - score) * (size / total_instance)
            
        return gini

    #Based on a attribute and threshold to split the data into left and right groups
    def split_dataset(self,dataset,attribute,threshold):
        left = []
        right = []
        for row in dataset:
            if row[attribute] >= threshold:
                right.append(row)
            else:
                left.append(row)
        return left,right

    #create the value of terminal node
    def to_terminal(self,child_node):
        outcomes = [row[-1] for row in child_node]
        return max(set(outcomes), key=outcomes.count)

    
    def _predict(self,node,data):
        if data[node['attribute']] >= node['threshold']:
            if isinstance(node['right'],dict):
                return self._predict(node['right'],data)
            else:
                return node['right']
        else:
            if isinstance(node['left'],dict):
                return self._predict(node['left'],data)
            else:
                return node['left']  
            
    #predict the y of data  
    #used to test the accuracy of this algorithm
    def predict(self,data):
        if self.tree['attribute'] >= self.tree['threshold']:
            if isinstance(self.tree['right'],dict):
                return self._predict(self.tree['right'],data)
            else:
                return self.tree['right']
        else:
            if isinstance(self.tree['left'],dict):
                return self._predict(self.tree['left'],data)
            else:
                return self.tree['left']
            
def load_csv(csv):
    data = pandas.read_csv(csv)
    return data.values.tolist()

def accuracy(tree,test_data):
    size = len(test_data)
    positive = 0
    for data in test_data:
#         print(data)
#         print("prediction: %r" % tree.predict(data))
#         print("GroundTruth: %r" %data[-1])
        #print (tree.predict(data))
        if tree.predict(data) == data[-1]:
            positive += 1
    return positive / size

if __name__ is '__main__':
    file_name = 'data_banknote_authentication.csv'
    dataset = load_csv('Data/Decision Tree/'+file_name)
    print(type(dataset))
    print(len(dataset))
    train_subset = dataset[0:1000]
    test_subset = dataset[1000:]
    tree = CARTree()
    max_depth = 5
    min_size = 3
    tree.learn(dataset,max_depth,min_size)
    print(tree.tree)
#     acc = accuracy(tree,dataset)
#     print("Accuracy:{}".format(acc))

<class 'list'>
1371
Total number of attributes is 5
Total number of attributes is 5
Total number of attributes is 5
Total number of attributes is 5
Total number of attributes is 5
Total number of attributes is 5
Total number of attributes is 5
{'right': {'right': 0.0, 'threshold': -4.3839, 'attribute': 2, 'left': 1.0, 'score': 0.1389514138988929}, 'threshold': 0.3223, 'attribute': 0, 'left': {'right': {'right': 0.0, 'threshold': -4.2859, 'attribute': 0, 'left': 1.0, 'score': 0.0}, 'threshold': 7.6274, 'attribute': 1, 'left': {'right': {'right': 0.0, 'threshold': 5.8974, 'attribute': 1, 'left': 1.0, 'score': 0.22892416225749565}, 'threshold': -0.39816, 'attribute': 0, 'left': {'right': 1.0, 'threshold': 6.2204, 'attribute': 2, 'left': 1.0, 'score': 0.06477884068623131}, 'score': 0.11743153350587424}, 'score': 0.15961960854754184}, 'score': 0.24696240682325166}


In [21]:
left = [0,1,2,3,4]
right = [5,6,7,9]
print(left+right)

[0, 1, 2, 3, 4, 5, 6, 7, 9]
